In [3]:
import torch
import torch.nn as nn
import os

import wget as wget
from tqdm.notebook import tqdm

print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 1.13.1


In [54]:
#!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
#!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
#!pip install torch-geometric
#!pip install ogb
#!pip install wget

In [55]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen
import os


def get_graphs(url, folder):
  """
  Inspired by: https://python.plainenglish.io/notesdownloader-use-web-scraping-to-download-all-pdfs-with-python-511ea9f55e48
  """
  if not os.path.exists(folder):
    os.makedirs(folder)

  links = []
  html = urlopen(url).read()
  html_page = bs(html, features="lxml") 

  for link in html_page.find_all('a'):
    current_link = link.get('href')

    if current_link.endswith('gz'):
      links.append(url + current_link)

  for link in tqdm(links):
    try:
      wget.download(link, out = folder)
    except:
      print("Unable to Download A File")
  print("File download done!")

In [56]:
import gzip
import shutil
from pathlib import Path


def uncompress_files(path, outpath):
  """
  Inspired by https://stackoverflow.com/questions/3548673/how-can-i-replace-or-strip-an-extension-from-a-filename-in-python
  and https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python
  """
  if not os.path.exists(outpath):
    os.makedirs(outpath)
  for file in tqdm(os.listdir(path)):
    if os.path.isdir(os.path.join(path, file)):
      continue
    new_filename = Path(file)
    extensions = "".join(new_filename.suffixes)
    new_ext = ".txt"
    new_filename = str(new_filename).replace(extensions, new_ext)
    new_filename = os.path.join(outpath, new_filename)

    with gzip.open(os.path.join(path, file), 'rb') as f_in:
        with open(new_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [57]:
#get_graphs("http://lime.cs.elte.hu/~kpeter/data/mcf/netgen/", './content/data/netgen')

In [58]:
get_graphs("http://lime.cs.elte.hu/~kpeter/data/mcf/road/", './data/road/compressed')

  0%|          | 0/70 [00:00<?, ?it/s]

File download done!


In [59]:
uncompress_files('./data/road/compressed', './data/raw')

  0%|          | 0/70 [00:00<?, ?it/s]

In [80]:
%run successive_shortest_paths.ipynb

number of edges: 24
Number of flow updates: 5, final flow value: 150


/home/alexandre/miniconda3/envs/cs224w/lib/python3.10/site-packages/nbformat/__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


ValueError: too many values to unpack (expected 2)

ValueError: too many values to unpack (expected 2)

In [164]:
def parse(filename) -> Network:
    """
    Parses a network file following the DIMACS problem specification
    structure and transforms it into a Network object

    Some elements of the specification:
    - Lines starting in c are comments
    - Lines starting in p explain what problem to solve (can be ignored,
      we only consider minimum-cost flow problems)
    - Lines starting in n define nodes
    - Lines starting in a define arcs (edges)

    Args:
        filename: name of the file containing the network data

    Returns:
        The corresponding Network object
    """
    # Lines we can ignore
    ignore_list = ['c', 'p']

    file = open(filename, 'r')

    # Nodes is a hashmap from node values to their supply
    nodes = {}
    # Edges is a hashmap from edges to a tuple with their capacity and cost
    edges = {}

    for line in file:
        if len(line) > 0 and line[0] not in ignore_list:
            if line[0] == 'n':
                # Node parsing
                node = [int(elem) for elem in line.split(' ')[1:]]
                nodes[node[0]] = node[1]
            elif line[0] == 'a':
                arc = [int(elem) for elem in line.split(' ')[1:]]
                node1 = arc[0]
                node2 = arc[1]
                capacity = arc[3]
                cost = arc[4]

                # Only nodes with non-zero supply are in a "node line"
                if node1 not in nodes:
                    nodes[node1] = 0
                if node2 not in nodes:
                    nodes[node2] = 0
                if (node1, node2) in edges:
                    # TODO not amazing (reaverages every time)
                    old_capacity, old_cost = edges[(node1, node2)]
                    new_cost = old_cost * old_capacity + cost * capacity
                    new_cost /= (old_capacity + capacity)
                    edges[(node1, node2)] = (old_capacity + capacity, new_cost)
                else:
                    edges[(node1, node2)] = (capacity, cost)
    file.close()

    capacities, costs = zip(*edges.values())
    network = Network(list(nodes.keys()), list(edges.keys()), capacities, costs, list(nodes.values()))
    #TODO data types?
    print(f"This dataset contains: {len(nodes.keys())} nodes and {len(edges.keys())} edges")
    if len(edges.keys()) <= 1e6:
        index = {node: index for node, index in zip(nodes, range(len(nodes)))}
        x = torch.tensor([supply for supply in nodes.values()]).reshape((-1, 1))
        edge_index = torch.reshape(torch.tensor([[index[e[0]], index[e[1]]] for e in edges]), (2, -1))
        edge_attr = torch.tensor([list(attributes) for attributes in edges.values()])
        print("starting to run successive shortest paths")
        iter_limit = 150
        converged, f, p = successive_shortest_paths(network, iter_limit = iter_limit)
        print("finished running successive shortest paths")
        y = torch.tensor(primal_value(network, p)).reshape(1,1)
        if converged:
            return {"converged": True, "x": x, "edge_index": edge_index, "edge_attr": edge_attr, "y": y}
    return {"converged": False}

In [82]:
parse("./resources/road_flow_01_DC_a.txt")

This dataset contains: 9559 nodes and 29682 edges
starting to run successive shortest paths
Current iteration number: 10, flow value: 6154440.0
Current iteration number: 20, flow value: 14143320.0
Current iteration number: 30, flow value: 26295300.0
Current iteration number: 40, flow value: 45820225.71428572
Current iteration number: 50, flow value: 64601545.71428572
Current iteration number: 60, flow value: 84710700.0
number of edges: 59396
Number of flow updates: 62, final flow value: 96478500.0
finished running successive shortest paths


(True,
 tensor([ 480,  720, -960,  ...,    0,    0,    0]),
 tensor([[  16,   17],
         [  17,   16],
         [  18,   16],
         ...,
         [9556, 9557],
         [9556, 9558],
         [9558, 9556]]),
 tensor([[480.,  42.],
         [480.,  42.],
         [120.,  60.],
         ...,
         [480., 452.],
         [480., 231.],
         [480., 231.]]),
 96478440.0)

In [89]:
parse("data/raw/road_flow_06_FL_d.txt")

This dataset contains: 1048497 nodes and 2629268 edges


AttributeError: type object 'Opt' has no attribute 'empty'

In [4]:
import os.path as osp

import torch
from torch_geometric.data import Data, Dataset, download_url


class MinCostDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        """If these files are found in the raw directory, download is skipped"""
        return []

    @property
    def processed_file_names(self):
        """If these files are found in the processed directory, processing is skipped"""
        processed_files = []
        path = self.processed_dir
        for file in tqdm(os.listdir(path)):
            file_path = os.path.join(path, file)
            if not os.path.isdir(file_path) and not file == "pre_filter.pt" and not file == "pre_transform.pt":
                processed_files.append(file)

        return processed_files

    def download(self):
        pass

    def process(self):
        idx = 0
        path = self.raw_dir
        for file in tqdm(os.listdir(path)):
            print(file)
            file_path = os.path.join(path, file)
            if os.path.isdir(file_path):
                continue
            # Read data from `raw_path`.
            output = parse(file_path)
            if output["converged"]:
                x = output["x"]
                edge_index = output["edge_index"]
                edge_attr = output["edge_attr"]
                y = output["y"]
                data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, filename = file_path)

                torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [5]:
dataset = MinCostDataset(root = "./data/")

  0%|          | 0/34 [00:00<?, ?it/s]

In [6]:
print(dataset)

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

MinCostDataset(32)


In [7]:
def i_messed_up(dataset, idx):
    data = dataset.get(idx)
    #data.edge_index = torch.reshape(data.edge_index, (2, -1))
    #data.edge_attr = torch.reshape(data.edge_attr, (-1, 2))
    #data.x = torch.reshape(data.x, (-1,1))
    data.x = data.x.type(torch.float32)
    torch.save(data, osp.join(dataset.processed_dir, f'data_{idx}.pt'))
for i in range(32):
    i_messed_up(dataset, i)